## Introdução à Metodologia de Previsão de Preço com LSTM e Indicadores Técnicos

Este código implementa uma metodologia moderna para previsão de preços de ações no curtíssimo prazo (intradia), utilizando redes neurais recorrentes (LSTM bidirecionais) em conjunto com uma ampla gama de indicadores técnicos clássicos. A abordagem combina princípios tradicionais da análise técnica com técnicas contemporâneas de aprendizado profundo.

> **Importante**: Esta implementação está sendo aplicada inicialmente à ação `VALE3.SA`, da mineradora Vale, como uma **prova de conceito (PoC)**. A escolha se deve à sua alta liquidez e relevância no mercado brasileiro. A intenção é, após validação da abordagem com esta ação, **escalar a metodologia para outras ações da bolsa** que apresentem perfil semelhante ou apresentem interesse estratégico.

A seguir, descrevemos os principais componentes da metodologia:

### 1. **Aquisição de Dados**
Utiliza-se a biblioteca `yfinance` para coletar dados intradiários (com granularidade de 1 minuto) da ação `VALE3.SA` por um período de 7 dias. Os dados incluem colunas como `Open`, `High`, `Low`, `Close` e `Volume`.

### 2. **Engenharia de Atributos Técnicos**
A função `add_technical_indicators` incorpora uma rica variedade de indicadores técnicos ao conjunto de dados, dentre os quais se destacam:

- **Indicadores de momentum**: RSI, Estocástico, Oscilador Awesome.
- **Indicadores de tendência**: MACD, ADX, CCI, Média Móvel Exponencial.
- **Indicadores de volatilidade**: Bandas de Bollinger, ATR.
- **Indicadores de volume**: OBV, VWAP, Índice de Acumulação/Distribuição.
- **Características de candlestick**: corpo, sombras e amplitude do candle.

Esses atributos auxiliam o modelo a captar padrões não triviais no comportamento dos preços.

### 3. **Normalização e Sequenciamento**
Os dados são normalizados com `MinMaxScaler` e, em seguida, organizados em sequências temporais (janelas deslizantes) com tamanho variável (ex. 24, 36, 60 minutos), de modo a alimentar a rede neural com informações históricas para prever o próximo valor de fechamento.

### 4. **Estrutura do Modelo**
O modelo preditivo é uma rede neural do tipo **Bidirectional LSTM**, composta por duas camadas LSTM (com possibilidade de dropout), seguida de uma camada densa para saída única (preço futuro). A bidirecionalidade permite à rede capturar relações tanto no sentido passado → futuro quanto futuro → passado.

### 5. **Treinamento e Avaliação**
O modelo é treinado com validação cruzada (via `validation_split`) e parada antecipada (`EarlyStopping`). Para cada combinação de hiperparâmetros, calcula-se o erro médio absoluto (MAE) e o erro quadrático médio da raiz (RMSE) com base em dados de teste.

### 6. **Otimização de Hiperparâmetros**
A busca por melhores combinações de hiperparâmetros é feita através de uma busca randomizada, onde múltiplas configurações são testadas automaticamente, e os resultados são registrados para identificação do melhor desempenho.

### 7. **Visualização dos Resultados**
Após o melhor modelo ser identificado, suas previsões são desnormalizadas e comparadas graficamente com os valores reais. O gráfico resultante mostra visualmente a capacidade preditiva do modelo.

---

Essa metodologia visa conciliar o conhecimento consagrado dos mercados financeiros (análise técnica) com as potencialidades do aprendizado de máquina sequencial, oferecendo uma base robusta para previsões em cenários de alta frequência e curto prazo.

O uso inicial da ação VALE3 permite validar a robustez da abordagem com um ativo de grande liquidez. Uma vez confirmada a eficácia, a arquitetura será reaproveitada e ajustada para outras ações brasileiras, com foco na escalabilidade e reprodutibilidade do pipeline.


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from ta.momentum import RSIIndicator
from ta.trend import MACD, EMAIndicator
from ta.volatility import BollingerBands
from ta.volume import VolumeWeightedAveragePrice
from ta.momentum import StochasticOscillator, AwesomeOscillatorIndicator
from ta.volume import VolumeWeightedAveragePrice, OnBalanceVolumeIndicator, AccDistIndexIndicator
from ta.trend import CCIIndicator, ADXIndicator, EMAIndicator
from ta.volatility import AverageTrueRange
import random

# --- Parâmetros fixos ---

symbol = 'VALE3.SA'
period = '7d'
interval = '1m'
SEQ_LENGTH_DEFAULT = 24
TEST_SIZE = 0.2
VALIDATION_SPLIT = 0.2

A função `add_technical_indicators` é responsável por enriquecer um `DataFrame` de preços de ações com uma ampla variedade de **indicadores técnicos clássicos**, comumente utilizados em análise técnica para tomada de decisão no mercado financeiro.

Essa função assume que o `DataFrame` de entrada (`df`) contém, pelo menos, as colunas: `Open`, `High`, `Low`, `Close`, e `Volume`.

### Visão Geral

- **Objetivo:** Adicionar colunas com indicadores técnicos calculados a partir dos preços históricos da ação.
- **Pré-condição:** A coluna `'Close'` deve existir no `DataFrame`. Caso contrário, uma exceção será lançada.
- **Tratamento de Erros:** Qualquer falha no cálculo dos indicadores resulta em uma exceção com mensagem detalhada.

---

### Indicadores Adicionados

Abaixo, listam-se os indicadores incluídos e seu propósito:

#### 1. **Indicadores de Momentum**

- `RSI`: Índice de Força Relativa com janela de 14 períodos.
- `Stoch_K` e `Stoch_D`: Oscilador Estocástico %K e sua média móvel %D.
- `Awesome_Oscillator`: Mede a força do momentum com duas médias móveis simples (5 e 34).

#### 2. **Indicadores de Tendência**

- `MACD`, `MACD_signal`, `MACD_diff`: Conjunto completo de MACD (12, 26, 9), incluindo linha de sinal e histograma.
- `CCI`: Commodity Channel Index com janela de 20 períodos.
- `ADX`, `ADX_pos`, `ADX_neg`: Índice Direcional Médio (ADX), e componentes positivo e negativo, com janela de 14 períodos.
- `EMA_20`: Média móvel exponencial de 20 períodos.

#### 3. **Indicadores de Volatilidade**

- `BB_upper`, `BB_middle`, `BB_lower`: Bandas de Bollinger (20 períodos, 2 desvios padrão).
- `ATR`: True Range Médio (Average True Range) com janela de 14 períodos.

#### 4. **Indicadores de Volume**

- `OBV`: On-Balance Volume.
- `AccDistIndex`: Índice de Acumulação/Distribuição.
- `VWAP`: Preço médio ponderado por volume, com janela de 14 períodos (calculado somente se todas as colunas requeridas estiverem presentes).

#### 5. **Características do Candle**

- `Candle_Body`: Corpo do candle (Close - Open).
- `Candle_Range`: Amplitude do candle (High - Low).
- `Upper_Shadow`: Sombra superior (High - max(Open, Close)).
- `Lower_Shadow`: Sombra inferior (min(Open, Close) - Low).

---

### Importância

A função é parte fundamental da **engenharia de atributos (feature engineering)** do pipeline de aprendizado de máquina, permitindo que o modelo LSTM receba não apenas a série temporal bruta de preços, mas também variáveis derivadas que capturam comportamento de tendência, momentum, reversões e pressões de compra/venda.

Estes indicadores enriquecem os dados com uma **representação multidimensional do contexto do preço**, melhorando a capacidade preditiva do modelo.



In [ ]:
# --- Função para adicionar indicadores técnicos ---
def add_technical_indicators(df):
    if 'Close' not in df.columns:
        raise ValueError("DataFrame não contém coluna 'Close'")

    try:
        df['RSI'] = RSIIndicator(close=df['Close'], window=14).rsi()

        stoch = StochasticOscillator(high=df['High'], low=df['Low'], close=df['Close'], window=14, smooth_window=3)
        df['Stoch_K'] = stoch.stoch()
        df['Stoch_D'] = stoch.stoch_signal()

        ao = AwesomeOscillatorIndicator(high=df['High'], low=df['Low'], window1=5, window2=34)
        df['Awesome_Oscillator'] = ao.awesome_oscillator()

        macd = MACD(close=df['Close'], window_slow=26, window_fast=12, window_sign=9)
        df['MACD'] = macd.macd()
        df['MACD_signal'] = macd.macd_signal()
        df['MACD_diff'] = macd.macd_diff()


        df['CCI'] = CCIIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=20).cci()

        adx = ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=14)
        df['ADX'] = adx.adx()
        df['ADX_pos'] = adx.adx_pos()
        df['ADX_neg'] = adx.adx_neg()

        df['EMA_20'] = EMAIndicator(close=df['Close'], window=20).ema_indicator()

        bb = BollingerBands(close=df['Close'], window=20, window_dev=2)
        df['BB_upper'] = bb.bollinger_hband()
        df['BB_middle'] = bb.bollinger_mavg()
        df['BB_lower'] = bb.bollinger_lband()


        df['ATR'] = AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close'], window=14).average_true_range()

        df['OBV'] = OnBalanceVolumeIndicator(close=df['Close'], volume=df['Volume']).on_balance_volume()
        df['AccDistIndex'] = AccDistIndexIndicator(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume']).acc_dist_index()

        if all(col in df.columns for col in ['High', 'Low', 'Close', 'Volume']):
            df['VWAP'] = VolumeWeightedAveragePrice(
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                volume=df['Volume'],
                window=14
            ).volume_weighted_average_price()
        else:
            df['VWAP'] = np.nan

        df['Candle_Body'] = df['Close'] - df['Open']
        df['Candle_Range'] = df['High'] - df['Low']
        df['Upper_Shadow'] = df['High'] - df[['Close', 'Open']].max(axis=1)
        df['Lower_Shadow'] = df[['Close', 'Open']].min(axis=1) - df['Low']

    except Exception as e:
        print(f"Erro ao calcular indicadores técnicos: {str(e)}")
        raise

    return df

Neste módulo, são definidas duas funções essenciais para a construção e avaliação de um modelo de aprendizado profundo do tipo LSTM Bidirecional, aplicado à previsão de séries temporais financeiras.

---

### 📌 Função: `create_sequences(data, seq_length)`

Essa função transforma os dados de entrada em **sequências temporais**, conforme necessário para o treinamento de modelos do tipo LSTM.

#### **Parâmetros:**
- `data`: Matriz NumPy com os dados normalizados, geralmente incluindo vários indicadores técnicos.
- `seq_length`: Quantidade de passos temporais considerados em cada sequência (janela deslizante).

#### **Retorno:**
- `X`: Conjunto de entradas com forma `(n_amostras, seq_length, n_features)`.
- `y`: Vetor de saídas com o valor da variável alvo (geralmente o preço de fechamento) no instante seguinte à sequência.

#### **Lógica:**
Para cada posição `i` no tempo:
- `X[i]` = subconjunto de `data` com `seq_length` linhas (passos temporais).
- `y[i]` = valor da variável alvo na posição `i + seq_length`.

Esse formato é ideal para treinar modelos que aprendem padrões temporais.

---

### 📌 Função: `train_and_evaluate(...)`

Essa função encapsula todo o processo de construção, treinamento, validação e avaliação do modelo preditivo.

#### **Parâmetros:**
- `X_train`, `y_train`: Dados de treino.
- `X_test`, `y_test`: Dados de teste.
- `params`: Dicionário de hiperparâmetros, incluindo:
  - `'lstm_units_1'`, `'lstm_units_2'`: Número de unidades LSTM em cada camada.
  - `'dropout'`: Taxa de dropout.
  - `'learning_rate'`: Taxa de aprendizado.
  - `'epochs'`: Número máximo de épocas.
  - `'batch_size'`: Tamanho do lote.
  - `'seq_length'`: Tamanho das sequências de entrada.
  - `'scaler'`: Objeto scaler usado para normalização.
- `n_features`: Número de colunas (indicadores) por amostra.

---

### 🔧 Arquitetura do Modelo

A função monta uma **rede neural recorrente** com a seguinte estrutura:

```text
Entrada (shape: seq_length x n_features)
↓
Camada LSTM bidirecional (1ª)
↓
Dropout
↓
Camada LSTM bidirecional (2ª)
↓
Dropout
↓
Camada Densa (1 unidade - saída contínua)
```

- **LSTM Bidirecional**: Permite que o modelo aprenda dependências temporais tanto do passado quanto do futuro da sequência.
- **Dropout**: Previne overfitting ao remover conexões aleatoriamente durante o treinamento.

---

### 🧪 Validação e Early Stopping

- **Validação**: 10% (ou valor definido em `VALIDATION_SPLIT`) do conjunto de treino é usado para validação durante o treinamento.
- **EarlyStopping**: Interrompe o treinamento se a perda de validação não melhorar após 10 épocas consecutivas, evitando overfitting.

---

### 🔄 Inversão da Escala

Como os dados são normalizados antes do treino, a função realiza a **inversão da normalização** para interpretar os resultados em escala original:

```python
def inverse_transform(scaler, data):
    dummy = np.zeros((len(data), n_features))
    dummy[:, 0] = data.flatten()
    return scaler.inverse_transform(dummy)[:, 0]
```

A inversão é feita apenas para a **primeira coluna**, que representa o valor de `Close`.

---

### 📊 Avaliação Final

A função retorna:

- `rmse`: Erro quadrático médio da previsão (em escala original).
- `mae`: Erro absoluto médio da previsão.
- `history`: Histórico do treinamento (valores de perda por época).
- `model`: Modelo treinado (objeto Keras).

---

### 💡 Observação

Essas funções foram projetadas para integrar um pipeline de aprendizado profundo aplicado inicialmente aos dados da **ação da VALE (VALE3.SA)**, como uma **prova de conceito (PoC)**. Após validação do desempenho, o mesmo modelo poderá ser generalizado para outras ações ou ativos financeiros, com base na robustez dos indicadores técnicos e da modelagem sequencial temporal.

---


In [ ]:
# --- Função para criar sequências ---
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length - 1):
        X.append(data[i:(i + seq_length), :])
        y.append(data[i + seq_length, 0])  # Close price na posição 0
    return np.array(X), np.array(y)

# --- Função para treinar e avaliar o modelo ---
def train_and_evaluate(X_train, y_train, X_test, y_test, params, n_features):  
    model = Sequential([
        Bidirectional(LSTM(params['lstm_units_1'], return_sequences=True), input_shape=(params['seq_length'], n_features)),
        Dropout(params['dropout']),
        Bidirectional(LSTM(params['lstm_units_2'])),
        Dropout(params['dropout']),
        Dense(1)
    ])
    
    optimizer = Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='mse')
    
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(
        X_train, y_train,
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        validation_split=VALIDATION_SPLIT,
        verbose=0,
        callbacks=[early_stop]
    )
    
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    # Função para inverter escala
    def inverse_transform(scaler, data):
        dummy = np.zeros((len(data), n_features))
        dummy[:, 0] = data.flatten()
        return scaler.inverse_transform(dummy)[:, 0]
    
    y_test_inv = inverse_transform(params['scaler'], y_test)
    test_pred_inv = inverse_transform(params['scaler'], test_pred)
    
    rmse = np.sqrt(mean_squared_error(y_test_inv, test_pred_inv))
    mae = mean_absolute_error(y_test_inv, test_pred_inv)
    
    return rmse, mae, history, model


Este trecho de código é responsável por coletar os dados históricos da ação, enriquecer com indicadores técnicos e preparar os dados em formato adequado para o treinamento de um modelo de aprendizado profundo.

---

### 📥 Coleta de Dados com o `yfinance`

```python
df = yf.download(symbol, period=period, interval=interval, progress=True)
df.columns = df.columns.droplevel(1)
```

- Utiliza a biblioteca `yfinance` para baixar dados históricos de preços do ativo definido pela variável `symbol`.
- `period` define o intervalo temporal (ex: `"2y"`).
- `interval` define a granularidade (ex: `"1d"`, `"1h"`).
- Algumas vezes os dados retornam com MultiIndex; `droplevel(1)` limpa isso.

---

### 📈 Enriquecimento com Indicadores Técnicos

```python
df = add_technical_indicators(df)
df.dropna(inplace=True)
```

- A função `add_technical_indicators(df)` insere colunas com indicadores de momentum, tendência, volatilidade, volume e padrões de candle.
- Após o cálculo dos indicadores, valores iniciais podem conter `NaN` devido às janelas móveis → são removidos com `dropna`.

---

### 🧠 Seleção das Features

```python
available_features = [ ... ]
features = [f for f in available_features if f in df.columns]
df = df[features]
```

- Define um conjunto completo de **variáveis de entrada candidatas** (features) para o modelo.
- Utiliza uma verificação para garantir que apenas as colunas calculadas de fato (que existem em `df`) sejam usadas.
- Garante consistência e evita erros caso alguma feature falhe durante o cálculo.

#### As features selecionadas incluem:
- **Preços e volume básicos**: `Open`, `High`, `Low`, `Close`, `Volume`.
- **Indicadores técnicos de momentum**: `RSI`, `Stoch_K`, `Stoch_D`, `Awesome_Oscillator`.
- **MACD e seus componentes**: `MACD`, `MACD_signal`, `MACD_diff`.
- **Indicadores de tendência**: `CCI`, `ADX`, `ADX_pos`, `ADX_neg`.
- **Bandas de Bollinger**: `BB_upper`, `BB_middle`, `BB_lower`.
- **Média móvel**: `EMA_20`.
- **Volatilidade**: `ATR`.
- **Volume inteligente**: `VWAP`, `OBV`, `AccDistIndex`.
- **Padrões de candle**: `Candle_Body`, `Candle_Range`, `Upper_Shadow`, `Lower_Shadow`.

---

### ✂️ Divisão Treino/Teste com Normalização

```python
def prepare_data(df, seq_length=SEQ_LENGTH_DEFAULT):
    num_samples = len(df) - seq_length - 1
    train_size = int(num_samples * (1 - TEST_SIZE))
    
    df_train = df.iloc[:train_size + seq_length + 1].copy()
    df_test = df.iloc[train_size:].copy()
```

- Define a proporção de treino/teste com base na constante `TEST_SIZE` (ex: 0.2).
- Garante que tanto o conjunto de treino quanto o de teste contenham sequências completas (`+ seq_length + 1`).

---

### 📊 Normalização com `MinMaxScaler`

```python
scaler = MinMaxScaler()
scaler.fit(df_train)
train_scaled = scaler.transform(df_train)
test_scaled = scaler.transform(df_test)
```

- Aplica a **normalização MinMax** nos dados, transformando os valores para a faixa `[0, 1]`.
- **Importante**: o scaler é treinado **apenas com dados de treino**, para evitar vazamento de informação (`data leakage`).

---

### 🔄 Criação das Sequências Temporais

```python
X_train, y_train = create_sequences(train_scaled, seq_length)
X_test, y_test = create_sequences(test_scaled, seq_length)
```

- Gera os conjuntos de entrada (`X`) e saída (`y`) para o modelo, com base nas janelas de `seq_length` períodos.

---

### 🔚 Retorno da Função

```python
return X_train, y_train, X_test, y_test, scaler
```

- A função retorna os dados processados e prontos para o treinamento, além do `scaler`, que será necessário para inverter a normalização após a predição.

---

### 🧪 Contexto da Prova de Conceito (PoC)

Este pipeline de preparação de dados foi desenvolvido no contexto de uma **prova de conceito (PoC)** uti


In [ ]:
df = yf.download(symbol, period=period, interval=interval, progress=True)
df.columns = df.columns.droplevel(1)

df = add_technical_indicators(df)
df.dropna(inplace=True)

available_features = [
    'Open', 'High', 'Low', 'Close', 'Volume',             # Preços e volume básicos
    'RSI', 'Stoch_K', 'Stoch_D', 'Awesome_Oscillator',    # Indicadores de momentum
    'MACD', 'MACD_signal', 'MACD_diff',                   # MACD
    'CCI', 'ADX', 'ADX_pos', 'ADX_neg',                   # Indicadores de tendência
    'BB_upper', 'BB_middle', 'BB_lower',                  # Bandas de Bollinger
    'EMA_20',                                             # Média móvel exponencial
    'ATR',                                                # Volatilidade
    'VWAP', 'OBV', 'AccDistIndex',                        # Indicadores de volume
    'Candle_Body', 'Candle_Range', 'Upper_Shadow', 'Lower_Shadow'  # Candlestick features
]

features = [f for f in available_features if f in df.columns]
print("Features selecionadas:", features)

df = df[features]

# --- Divisão treino/teste ---
def prepare_data(df, seq_length=SEQ_LENGTH_DEFAULT):
    num_samples = len(df) - seq_length - 1
    train_size = int(num_samples * (1 - TEST_SIZE))
    df_train = df.iloc[:train_size + seq_length + 1].copy()
    df_test = df.iloc[train_size:].copy()
    
    scaler = MinMaxScaler()
    scaler.fit(df_train)
    train_scaled = scaler.transform(df_train)
    test_scaled = scaler.transform(df_test)
    
    X_train, y_train = create_sequences(train_scaled, seq_length)
    X_test, y_test = create_sequences(test_scaled, seq_length)
    return X_train, y_train, X_test, y_test, scaler

## 🔧 Busca Randomizada de Hiperparâmetros para Modelos LSTM

Este bloco de código realiza uma **busca aleatória (random search)** para encontrar a melhor combinação de hiperparâmetros de um modelo LSTM bidirecional, visando minimizar o erro RMSE (Root Mean Squared Error) na previsão de preços de ativos financeiros.

---

### 🎯 Objetivo da Otimização

Selecionar, entre diferentes combinações de hiperparâmetros, aquela que oferece o melhor desempenho em termos de RMSE sobre o conjunto de teste. O processo explora o espaço de configurações e compara os resultados, armazenando o melhor modelo encontrado.

---

### 🧰 Definição do Espaço de Busca

```python
search_space = {
    'seq_length': [12, 24, 36, 48, 60],
    'batch_size': [16, 32, 64, 128],
    'epochs': [50, 80, 100, 120, 150],
    'lstm_units_1': [64, 128, 256, 384, 512],
    'lstm_units_2': [32, 64, 128, 192, 256],
    'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
    'learning_rate': [0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005]  
}
```

- Define os valores possíveis para cada hiperparâmetro.
- **`seq_length`**: Número de observações anteriores usadas como entrada.
- **`batch_size`**: Quantidade de amostras processadas por iteração.
- **`epochs`**: Número máximo de ciclos de treinamento.
- **`lstm_units_1`/`lstm_units_2`**: Número de neurônios nas camadas LSTM bidirecionais.
- **`dropout`**: Fração de unidades descartadas durante o treinamento para evitar overfitting.
- **`learning_rate`**: Taxa de aprendizado usada pelo otimizador `Adam`.

---

### 🔁 Processo de Busca Aleatória

```python
n_trials = 10
best_rmse = float('inf')
```

- Define `n_trials` como o número total de execuções (combinações aleatórias a serem testadas).
- Inicializa a variável `best_rmse` como infinito, para garantir que qualquer RMSE válido será menor.

---

### 📦 Loop de Execuções

```python
for trial in range(n_trials):
    params = {
        'seq_length': random.choice(...),
        ...
    }
```

- Para cada tentativa:
  - Seleciona **aleatoriamente** um valor para cada hiperparâmetro a partir do `search_space`.
  - Armazena os valores selecionados no dicionário `params`.

---

### 📊 Preparação de Dados com `prepare_data`

```python
X_train, y_train, X_test, y_test, scaler = prepare_data(df, seq_length=params['seq_length'])
params['scaler'] = scaler
```

- Os dados são normalizados e divididos com base no `seq_length` escolhido.
- O `scaler` é salvo dentro de `params` para permitir a inversão da escala posteriormente.

---

### 🏋️‍♂️ Treinamento e Avaliação

```python
rmse, mae, history, model = train_and_evaluate(...)
```

- Treina o modelo LSTM com os hiperparâmetros atuais.
- Avalia seu desempenho utilizando métricas padronizadas:
  - **RMSE (Root Mean Squared Error)**: penaliza erros maiores.
  - **MAE (Mean Absolute Error)**: média dos erros absolutos.

---

### ✅ Atualização do Melhor Modelo

```python
if rmse < best_rmse:
    best_rmse = rmse
    best_params = params
    best_model = model
    best_history = history
```

- Se o modelo atual tiver desempenho superior ao melhor até então:
  - Salva os hiperparâmetros, o modelo treinado e o histórico de treinamento.

---

### 🧾 Registro dos Resultados

```python
results.append({
    'trial': trial + 1,
    'rmse': rmse,
    'mae': mae,
    **params
})
```

- Armazena todas as métricas e os hiperparâmetros testados para posterior análise.

---

### 🏁 Resultados Finais

```python
print("\nMelhor RMSE:", best_rmse)
print("Melhores hiperparâmetros:", best_params)
```

- Exibe a melhor configuração encontrada após todos os testes realizados.

---

### 🧠 Considerações Técnicas

- **Random Search** é menos eficiente que métodos baseados em gradiente ou Bayesian Optimization, mas útil em contextos onde:
  - O custo de cada avaliação é alto.
  - O espaço de busca é não contínuo.
  - A função objetivo (modelo) é não diferenciável.

- A randomização evita viés e permite cobrir melhor o espaço de busca do que uma simples busca em grade (grid search) quando o número de tentativas é limitado.

- Este método é especialmente adequado para provas de conceito (PoC) e cenários com restrições de tempo computacional.

---

### 📌 Relevância para Séries Temporais

Em modelos baseados em LSTM para séries temporais:
- Pequenas variações em `seq_length`, `lstm_units` ou `learning_rate` podem ter impacto significativo na capacidade do modelo de **capturar padrões de longo prazo**.
- O uso de `Dropout` ajuda a combater o overfitting, comum quando há correlação temporal elevada entre amostras adjacentes.

---

### 🔬 Próximos Passos

- Persistir os melhores resultados (parâmetros, métricas, modelo).
- Visualizar o `history` de treinamento (loss x epochs).
- Avaliar generalização para novos dados (validação out-of-sample).
- Implementar uma busca **Bayesiana** com `Optuna` ou `KerasTuner` para refinamento posterior.

---


In [ ]:
# --- Espaço de hiperparâmetros para busca ---
search_space = {
    'seq_length': [12, 24, 36, 48, 60],
    'batch_size': [16, 32, 64, 128],
    'epochs': [50, 80, 100, 120, 150],
    'lstm_units_1': [64, 128, 256, 384, 512],
    'lstm_units_2': [32, 64, 128, 192, 256],
    'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
    'learning_rate': [0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005]  
}

# --- Busca randomizada ---
n_trials = 10  # quantas combinações testar
best_rmse = float('inf')
best_params = None
best_model = None
best_history = None
results = []

for trial in range(n_trials):
    params = {
        'seq_length': random.choice(search_space['seq_length']),
        'batch_size': random.choice(search_space['batch_size']),
        'epochs': random.choice(search_space['epochs']),
        'lstm_units_1': random.choice(search_space['lstm_units_1']),
        'lstm_units_2': random.choice(search_space['lstm_units_2']),
        'dropout': random.choice(search_space['dropout']),
        'learning_rate': random.choice(search_space['learning_rate']),
    }
    
    print(f"\nTrial {trial+1}/{n_trials} com params: {params}")
    
    # Preparar dados para o seq_length atual
    X_train, y_train, X_test, y_test, scaler = prepare_data(df, seq_length=params['seq_length'])
    params['scaler'] = scaler
    
    # Treinar e avaliar
    try:
        rmse, mae, history, model = train_and_evaluate(X_train, y_train, X_test, y_test, params, n_features=len(features))
        print(f"RMSE: {rmse:.4f} - MAE: {mae:.4f}")
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = params
            best_model = model
            best_history = history
    except Exception as e:
        print(f"Erro no treino/avaliação: {e}")
        
results.append({
    'trial': trial + 1,
    'rmse': rmse,
    'mae': mae,
    **params  # isso inclui todos os hiperparâmetros testados
})

print("\nMelhor RMSE:", best_rmse)
print("Melhores hiperparâmetros:", best_params)

| O código plota um gráfico que compara os preços reais de fechamento com as previsões feitas pelo melhor modelo treinado. Para isso, ele primeiro reverte a escala dos dados normalizados para valores reais, tanto para os dados de teste quanto para as previsões, utilizando o mesmo scaler. Em seguida, exibe essa comparação visualmente, permitindo avaliar a precisão do modelo na previsão dos preços ao longo do tempo.

In [ ]:
# --- Plotar resultado com melhor modelo ---
if best_model:
    X_train, y_train, X_test, y_test, scaler = prepare_data(df, seq_length=best_params['seq_length'])
    y_test_inv = scaler.inverse_transform(
        np.concatenate([y_test.reshape(-1,1), np.zeros((len(y_test), len(features)-1))], axis=1)
    )[:, 0]
    
    test_pred = best_model.predict(X_test)
    test_pred_inv = scaler.inverse_transform(
        np.concatenate([test_pred, np.zeros((len(test_pred), len(features)-1))], axis=1)
    )[:, 0]
    
    plt.figure(figsize=(16, 8))
    plt.plot(y_test_inv, label='Valor Real')
    plt.plot(test_pred_inv, label='Previsão')
    plt.title(f'Previsão do Preço de Fechamento - {symbol} (Melhor Modelo)')
    plt.xlabel('Horas')
    plt.ylabel('Preço (R$)')
    plt.legend()
    plt.show()

| Após o treinamento do modelo e a obtenção dos melhores resultados, é fundamental salvar o modelo para que ele possa ser reutilizado posteriormente sem a necessidade de ser re-treinado. O formato .h5 é amplamente utilizado em projetos que envolvem modelos Keras/TensorFlow, pois permite armazenar tanto a arquitetura quanto os pesos do modelo em um único arquivo. Isso facilita a implantação, compartilhamento e posterior análise do modelo.

In [ ]:
# Salvando o melhor modelo em formato HDF5 (.h5)
if best_model:
    best_model.save('../model/modelo_v1.h5')
    print("Modelo salvo com sucesso no arquivo 'best_lstm_model.h5'")
